In [1]:
from preprocess import *
import numpy as np
import wandb
import torch
import pandas as pd
import os

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(DEVICE)

cuda


In [3]:
# wrun = wandb.init()

wrun = wandb.Api().artifact('shu7bh/ELMO/best_model_3y4z0vh5:v0', type='model')
# artifact = wrun.use_artifact('shu7bh/ELMO/best_model_3y4z0vh5:v0', type='model')
# artifact_dir = artifact.download()
artifact_dir = wrun.download()
cfg = wandb.Api().run('shu7bh/ELMO/3y4z0vh5').config# 

# import wandb
# wrun = wandb.init()
# artifact = wrun.use_artifact('shu7bh/ELMO/best_model_3y4z0vh5:v0', type='model')
# cfg = wrun.config
# artifact_dir = artifact.download()

wandb: Downloading large artifact best_model_3y4z0vh5:v0, 569.43MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.3


In [4]:
cfg

{'epochs': 100,
 'dropout': 0,
 'optimizer': 'Adam',
 'hidden_dim': 1000,
 'num_layers': 2,
 'dev_train_len': 25000,
 'embedding_dim': 100,
 'learning_rate': 0.001,
 'dev_validation_len': 5000}

In [5]:
DEV_TRAIN_LEN = cfg['dev_train_len']
DEV_VALIDATION_LEN = cfg['dev_validation_len']
HIDDEN_DIM = cfg['hidden_dim']
DROP_OUT = cfg['dropout']
LEARNING_RATE = cfg['learning_rate']
EPOCHS = cfg['epochs']
EMBEDDITNG_DIM = cfg['embedding_dim']
NUM_LAYERS = cfg['num_layers']
BATCH_SIZE = 16
if HIDDEN_DIM in [300, 500]:
    BATCH_SIZE = 32

DIR = '/scratch/shu7bh/RES/PRE'

print(DEV_TRAIN_LEN)
print(DEV_VALIDATION_LEN)
print(HIDDEN_DIM)
print(DROP_OUT)
print(LEARNING_RATE)
print(EPOCHS)
print(EMBEDDITNG_DIM)
print(NUM_LAYERS)
print(BATCH_SIZE)
print(DIR)

25000
5000
1000
0
0.001
100
100
2
16
/scratch/shu7bh/RES/PRE


In [6]:
import os
if not os.path.exists(DIR):
    os.makedirs(DIR)

In [7]:
print(DEV_TRAIN_LEN)
print(DEV_VALIDATION_LEN)

25000
5000


In [8]:
df = pd.read_csv('data/train.csv')
df = df.sample(frac=1, random_state=0).reset_index(drop=True)
df['Description'] = df['Description'].apply(tokenize_corpus)
df['Description'] = df['Description'].apply(get_word_tokenized_corpus)

In [9]:
# dev_train = df[:DEV_TRAIN_LEN]['Description']
dev_validation = df[DEV_TRAIN_LEN:DEV_TRAIN_LEN + DEV_VALIDATION_LEN]['Description']

In [10]:
dev_validation

25000    [linux, is, gaining, ground, with, companies, ...
25001    [google, ,, the, internet, search, engine, ,, ...
25002    [ap, defending, champion, tiger, woods, said, ...
25003    [advanced, micro, devices, will, add, quot, ;,...
25004    [reuters, about, 100, tibetan, exiles, chanted...
                               ...                        
29995    [they, were, 4, 12, last, year, ., they, have,...
29996    [luke, donald, and, paul, casey, are, unlikely...
29997    [yahoo, is, eager, to, improve, its, web, base...
29998    [ankara, strasbourg, ,, france, reuters, prime...
29999    [leipzig, game, convention, in, germany, ,, th...
Name: Description, Length: 5000, dtype: object

In [11]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from dataset import SentencesDataset
from elmo import ELMO
from torch import nn

In [12]:
class Collator:
    def __init__(self, Emb):
        self.pad_index = Emb.key_to_index['<pad>']

    def __call__(self, batch):
        X, X_lengths = zip(*batch)
        X = pad_sequence(X, batch_first=True, padding_value=self.pad_index)
        return X[:, :-1], X[:, 1:], torch.stack(X_lengths) - 1

In [13]:
import tqdm

def run_epoch(model, dataloader, loss_fn):
    model.eval()
    epoch_loss = []

    pbar = tqdm.tqdm(dataloader)

    for X, Y, X_lengths in pbar:
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)

        Y_pred = model(X, X_lengths)
        Y_pred = Y_pred.reshape(-1, Y_pred.shape[2])

        Y = Y.reshape(-1)

        loss = loss_fn(Y_pred, Y)
        epoch_loss.append(loss.item())

        pbar.set_description(f'Loss: {loss.item():7.4f}, Avg Loss: {np.mean(epoch_loss):7.4f}')

    return np.mean(epoch_loss)

In [14]:
def validate(elmo, validation_dataloader, loss_fn):
    with torch.no_grad():
        epoch_loss = run_epoch(elmo, validation_dataloader, loss_fn)
        print(f'Validation Loss: {epoch_loss:7.4f}')

In [110]:
Emb = create_vocab(df['Description'], EMBEDDITNG_DIM)

# dev_train_dataset = SentencesDataset(dev_train, Emb)
dev_validation_dataset = SentencesDataset(dev_validation, Emb)

collate_fn = Collator(Emb)

# training_dataloader = DataLoader(dev_train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)
validation_dataloader = DataLoader(dev_validation_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)

In [111]:
print(Emb[10])

[-0.73088   0.60521   0.53377  -0.30728   0.10999  -0.39782   0.25639
  0.85523  -0.51615  -0.1276   -0.052802  0.53213  -0.070358  0.5007
  0.67241  -0.75289  -0.059124 -0.15873  -0.27048  -0.82119  -0.46997
  0.24722   0.022606  0.32347  -0.2085   -0.28828   0.062202  0.29071
 -0.28597  -0.54406   0.59047  -0.76898   0.36901  -0.63082   0.37619
 -0.092507  0.32857  -0.16039  -0.24477  -0.12558  -0.075998  0.10752
 -0.70408  -0.45696   0.69928   1.1006   -0.096052  0.073672 -0.41376
  0.93381  -0.44251   0.66334   0.28542   0.31271  -0.31862  -0.19592
  0.13893  -0.56505   0.11214   0.7031   -0.96368   0.58617   0.88596
  0.67802   0.81865   0.37298   0.70602  -0.38213   1.1708   -0.83419
  0.33654   0.42369   0.82828   0.060616 -0.11314  -0.78854   0.10705
  0.34057   0.98908   0.80425  -0.12698   0.18206  -0.7656    0.42928
 -0.072067  0.19472  -0.30368  -0.4751    0.10632   0.19623   0.17147
 -0.013407 -0.97829  -0.33786   0.34114   0.8992   -0.90568  -0.15093
  0.30339   0.42763 ]

In [112]:
elmo = ELMO(Emb, HIDDEN_DIM, DROP_OUT, NUM_LAYERS)

In [113]:
print(Emb[10])
# print(Emb.key_to_index['house'])

[-0.73088   0.60521   0.53377  -0.30728   0.10999  -0.39782   0.25639
  0.85523  -0.51615  -0.1276   -0.052802  0.53213  -0.070358  0.5007
  0.67241  -0.75289  -0.059124 -0.15873  -0.27048  -0.82119  -0.46997
  0.24722   0.022606  0.32347  -0.2085   -0.28828   0.062202  0.29071
 -0.28597  -0.54406   0.59047  -0.76898   0.36901  -0.63082   0.37619
 -0.092507  0.32857  -0.16039  -0.24477  -0.12558  -0.075998  0.10752
 -0.70408  -0.45696   0.69928   1.1006   -0.096052  0.073672 -0.41376
  0.93381  -0.44251   0.66334   0.28542   0.31271  -0.31862  -0.19592
  0.13893  -0.56505   0.11214   0.7031   -0.96368   0.58617   0.88596
  0.67802   0.81865   0.37298   0.70602  -0.38213   1.1708   -0.83419
  0.33654   0.42369   0.82828   0.060616 -0.11314  -0.78854   0.10705
  0.34057   0.98908   0.80425  -0.12698   0.18206  -0.7656    0.42928
 -0.072067  0.19472  -0.30368  -0.4751    0.10632   0.19623   0.17147
 -0.013407 -0.97829  -0.33786   0.34114   0.8992   -0.90568  -0.15093
  0.30339   0.42763 ]

In [114]:
# run.file("best_model.pth").download(replace=True, root=DIR)

In [115]:
artifact_dir

'./artifacts/best_model_3y4z0vh5:v0'

In [116]:
elmo.load_state_dict(torch.load(os.path.join(DIR, 'best_model.pth')))
# elmo = elmo.to(DEVICE)
elmo_embeddings = list(elmo.parameters())[0].detach().cpu()
elmo.embedding = nn.Embedding.from_pretrained(elmo_embeddings.to(DEVICE))

In [117]:
print(elmo_embeddings.shape)
print(elmo_embeddings[10])

torch.Size([55421, 100])
tensor([ 0.2695, -0.4133,  0.2555,  0.0189, -0.0369,  0.6693, -0.4759, -0.4315,
         0.7589,  0.3562, -0.8274,  0.0919,  0.1156, -0.0347, -0.2910,  0.2723,
        -0.3793, -0.5807, -0.2133,  0.1883,  0.3747, -0.1412,  0.3004,  0.2402,
         0.2768,  0.5330, -0.8345,  0.0388,  0.4195,  0.3293,  0.5797,  0.0309,
        -0.1891, -0.4507, -0.2105, -0.4584, -0.0202, -1.0074, -0.4165,  0.7901,
        -1.1021,  0.2426, -0.0202, -0.5234,  0.5298,  0.5191,  0.3533,  0.1693,
        -0.3535, -0.6972,  0.5551,  0.3834, -0.1444,  0.6576,  0.5679, -1.4603,
         0.6166, -0.3619,  0.8611,  0.4859, -0.0629,  0.5049, -0.2980, -0.7418,
         0.2815,  0.3432,  0.5404,  0.2026,  0.2985, -0.8323, -0.2582, -0.4582,
        -0.2183, -0.5529,  0.3991,  0.1461,  0.2010,  0.4561, -0.5835,  0.6638,
         0.8766, -0.3691, -0.6414,  0.1002, -0.8823, -0.4552,  0.4315,  0.6305,
         0.0827, -0.3868,  0.2525,  0.0592,  0.2632, -0.3143, -0.1943,  0.6264,
        -0.0497

In [84]:
print(Emb[0])

[-0.35962    0.40762    0.67584   -1.4254    -0.25645    0.34921
  0.5475    -0.31423   -1.3885    -0.59486    0.036926  -0.77826
 -0.17036    0.29913    0.43609   -0.35711    0.24703    0.76785
 -0.19345    0.37874    0.59231   -0.28      -0.40628   -0.35337
  0.64482   -0.15114   -0.030707  -0.34519    0.062968   0.14298
 -0.80045    0.55164   -0.34825   -0.39483   -0.56737    0.13307
 -0.6018     0.53523   -0.41897   -0.16668    0.97426   -0.66592
  0.60532   -0.54342    0.66173    0.21875    0.89684    0.008652
  0.8836    -0.32895   -0.53328    0.23677   -0.75438    0.16423
 -0.2293    -0.41585    0.11206    0.20916    0.48118    0.87651
 -0.8563    -0.29795   -1.0721    -0.1228     0.80495   -0.28747
  0.4002     0.020672  -0.85426    0.082154  -0.0070616  0.25759
 -1.1279    -1.0936    -0.21632    0.22287   -0.082762   0.13074
 -0.48821    0.38517   -0.27818    0.035566  -0.79963   -0.13705
  0.41555   -0.52584   -0.24059    0.89586   -0.33438    0.83941
 -0.35232   -0.76319   -

In [85]:
Emb['the']

array([-0.81258 ,  0.050847, -0.093119,  0.11473 ,  0.042634,  0.44399 ,
       -0.49399 ,  0.058819, -0.13187 ,  0.64735 , -0.19067 , -0.61551 ,
        0.16782 , -0.62521 ,  0.35184 , -0.059459,  0.13065 ,  0.17959 ,
        1.1526  , -0.79385 ,  0.22522 , -0.34393 , -0.3346  , -0.41321 ,
        0.33249 ,  0.022747,  0.4358  ,  0.091137, -0.36174 , -0.16166 ,
        0.23005 ,  0.018134, -0.29529 , -0.010631, -0.25594 , -0.093366,
       -0.005642,  0.65419 ,  0.22336 , -0.083446,  0.036179, -0.82258 ,
        0.38452 ,  0.28986 , -0.83104 , -0.44402 ,  0.56309 ,  0.38907 ,
        0.060224, -1.1119  ,  0.2804  , -0.091257,  0.055767,  0.388   ,
       -0.061242, -0.51081 ,  0.91529 ,  0.34623 ,  0.69628 , -0.27054 ,
       -0.21503 ,  1.0313  , -0.23077 ,  0.33464 ,  0.28704 , -0.14482 ,
        1.1685  , -0.16915 , -0.17133 , -0.92822 ,  0.22395 ,  0.048626,
        0.51446 ,  0.39898 ,  0.22265 ,  0.24382 ,  0.55618 , -0.092959,
        0.60281 , -0.73542 , -0.42421 ,  0.30127 , 

In [86]:
elmo = elmo.to(DEVICE)

In [87]:
loss_fn = nn.CrossEntropyLoss(ignore_index=Emb.key_to_index['<pad>'])
validate(elmo, validation_dataloader, loss_fn)

Loss:  7.8234, Avg Loss:  8.0386: 100%|██████████| 313/313 [00:11<00:00, 28.42it/s]

Validation Loss:  8.0386


In [25]:
print(elmo)
print(elmo.fc.weight)

ELMO(
  (embedding): Embedding(55421, 100, padding_idx=55418)
  (lstm): LSTM(100, 1000, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=2000, out_features=55421, bias=True)
)
Parameter containing:
tensor([[ 0.0405,  0.0829,  0.0291,  ..., -0.1101, -0.0169, -0.0765],
        [ 0.1528,  0.1052,  0.0153,  ...,  0.1032, -0.2814, -0.1232],
        [ 0.0697,  0.0530,  0.0369,  ...,  0.3767, -0.1765, -0.0847],
        ...,
        [-0.0215,  0.1068,  0.0530,  ...,  0.1456, -0.0598, -0.0284],
        [ 0.0169,  0.1081,  0.0448,  ...,  0.1425, -0.0554, -0.0311],
        [ 0.2552, -0.0233, -0.0808,  ...,  0.1702,  0.1520,  0.0198]],
       device='cuda:0', requires_grad=True)


In [26]:
Emb['the']

array([-0.81258 ,  0.050847, -0.093119,  0.11473 ,  0.042634,  0.44399 ,
       -0.49399 ,  0.058819, -0.13187 ,  0.64735 , -0.19067 , -0.61551 ,
        0.16782 , -0.62521 ,  0.35184 , -0.059459,  0.13065 ,  0.17959 ,
        1.1526  , -0.79385 ,  0.22522 , -0.34393 , -0.3346  , -0.41321 ,
        0.33249 ,  0.022747,  0.4358  ,  0.091137, -0.36174 , -0.16166 ,
        0.23005 ,  0.018134, -0.29529 , -0.010631, -0.25594 , -0.093366,
       -0.005642,  0.65419 ,  0.22336 , -0.083446,  0.036179, -0.82258 ,
        0.38452 ,  0.28986 , -0.83104 , -0.44402 ,  0.56309 ,  0.38907 ,
        0.060224, -1.1119  ,  0.2804  , -0.091257,  0.055767,  0.388   ,
       -0.061242, -0.51081 ,  0.91529 ,  0.34623 ,  0.69628 , -0.27054 ,
       -0.21503 ,  1.0313  , -0.23077 ,  0.33464 ,  0.28704 , -0.14482 ,
        1.1685  , -0.16915 , -0.17133 , -0.92822 ,  0.22395 ,  0.048626,
        0.51446 ,  0.39898 ,  0.22265 ,  0.24382 ,  0.55618 , -0.092959,
        0.60281 , -0.73542 , -0.42421 ,  0.30127 , 